In [18]:
 
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig, default_data_collator
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from datasets import load_dataset
import os
from dotenv import load_dotenv
from huggingface_hub import login
import torch
from torch.utils.data import DataLoader
from IPython.display import Markdown, display



In [2]:
def get_private_keys(key_name):
    load_dotenv()
    key = os.getenv(key_name)
    if not key:
        raise ValueError(f"Missing key for {key_name} in .env") 
    return key

huggingface_hub_token = get_private_keys("huggingface_hub")
login(token=huggingface_hub_token)

In [3]:
#Base Model

model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  #or "fp4"
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
# Fine-tuned model
new_model_name = "tinyllama-v2-qa-343samples-finetuned-08082025"

tmp_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

tuned_model = PeftModel.from_pretrained(tmp_model, new_model_name)
tuned_model = tuned_model.merge_and_unload().eval()

/home/mrosaria/Projects/NLP/GymRat/ratenv/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [21]:
# Merge 'instruction' and 'output' as prompt and target
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['instruction'], batch['output'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens


# Merge 'instruction' and 'output' as prompt and target
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    return {"text": prompt}

In [24]:
input_json_file_name = "cleaned_1506_raw_outputs_samples1260_nreps3_new_tok256_chuck_size250_overlap30_.json"
dataset = load_dataset("json", data_files=input_json_file_name)


In [7]:
splits = dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset = splits["train"]
test_dataset = splits["test"]

In [8]:
test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=["instruction", "output"])

In [9]:
eval_ds = test_dataset.with_format('torch')


In [14]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [15]:
@torch.no_grad()
def compute_perplexity(model):
    losses = []
    
    for batch in eval_loader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        loss = model(**batch).loss
        losses.append(loss.item())

    return math.exp(sum(losses) / len(losses))

In [17]:
import math
print(f'Base Model Perplexity: {compute_perplexity(model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')


Base Model Perplexity: 8275.61
Tuned Model Perplexity: 8045.69


In [25]:
test_text = dataset["train"].train_test_split(test_size=0.1, seed=42)['test']


In [26]:
test_text = test_text.map(format_example)
print(test_text[1])

{'instruction': 'Stand up straight, with your feet shoulder-width apart, and your hands on your hips. Look straight ahead with your eyes closed. Take a deep breath in through your nose, and exhale through your mouth. Try to maintain this position for at least 10-15 seconds, then gradually release your arms and relax your shoulders?.', 'output': 'Your spine is aligned with your shoulders, and you are not tense or stiff. You can take a deep breath and relax your shoulders without having to engage them in any specific way. Your posture is good.', 'text': '### Instruction:\nStand up straight, with your feet shoulder-width apart, and your hands on your hips. Look straight ahead with your eyes closed. Take a deep breath in through your nose, and exhale through your mouth. Try to maintain this position for at least 10-15 seconds, then gradually release your arms and relax your shoulders?.\n\n### Response:\nYour spine is aligned with your shoulders, and you are not tense or stiff. You can take

In [43]:
def generate_response(model, model_tuned , message, output):
    prompt = [{"role": "user", "content": message}]
    print("Input Message:")
    display(Markdown(message))
    print("Output")
    display(Markdown(output))
    formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

    outputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")  # or "cpu"
    response = model.generate(
        **outputs,
        max_new_tokens=256,
        do_sample=True,         # enables sampling for variability
        temperature=0.7,       # controls randomness
        top_p=0.9,             # nucleus sampling
        top_k=50                # limits the number of highest probability vocabulary tokens to keep for generation
    )
    decoded =tokenizer.decode(response[0], skip_special_tokens=True)
    print(f"******************************")
    print(f"Response Base Model")

    display(Markdown(decoded.split('<|assistant|>')[-1]))
    print(f"******************************")
    
    response_tuned = model_tuned.generate(
        **outputs,
        max_new_tokens=256,
        do_sample=True,         # enables sampling for variability
        temperature=0.7,       # controls randomness
        top_p=0.9,             # nucleus sampling
        top_k=50                # limits the number of highest probability vocabulary tokens to keep for generation
    )
    decoded_tuned =tokenizer.decode(response_tuned[0], skip_special_tokens=True)
    print(f"Response Finetunned Model")
    display(Markdown(decoded_tuned.split('<|assistant|>')[-1]))
    print(f"********************************")
    return decoded, decoded_tuned

#_, _ = generate_response(model, tuned_model, test_text[2]['instruction'])


In [45]:
for i in range(10):
    generate_response(model, tuned_model, test_text[i]['instruction'], test_text[i]['output'])


Input Message:


Quick solutions for iliotibial band syndrome in runners. Involves the functional anatomy of the iliotibial band during flexion and extension of the knee. Biomechanics of the iliotibial band friction syndrome in runners. The functional anatomy of the iliotibial band during flexion and extension of the knee?.

Output


Involves the functional anatomy of the iliotibial band during flexion and extension of the knee. Biomechanics of the iliotibial band friction syndrome in runners. Quick solutions for iliotibial band syndrome in runners.

******************************
Response Base Model


...

******************************
Response Base Model



Yes, there is a specific standing position that causes pain in the low back and shoulders, and it is called the "sit-to-stand" position. This standing position involves standing with your feet shoulder-width apart and your hands on your thighs. If you experience pain in this position, it could be a sign that your lower back is not well-supported or that you may be overweight or overextending your lower back. This could also indicate that you have a sedentary lifestyle or a lack of exercise. To modify this position to alleviate pain, try using a strap or block to support your legs and feet while standing. Additionally, you may need to modify your sitting or standing posture to prevent further strain on your lower back.

******************************
Response Finetunned Model



Response:

Yes, there is a specific standing position that can cause pain in the low back or shoulders. This is called the "kneeling position," also known as the "squat." When we sit or stand in a kneeling position, our hips and thighs are parallel to the ground, which can put pressure on the lower back and shoulders. This can lead to discomfort or pain in these areas.

To modify your symptoms, you can try taking a step or two forward instead of straight ahead. This can help reduce pressure on the low back and shoulders. Alternatively, you could try doing some low-impact exercises, such as swimming or cycling, to reduce the pressure on the lower back and shoulders.

Overall, it's essential to understand the physiology of your body and how it responds to different standing positions. By understanding this, you can make informed choices about how you sit, stand, or move throughout the day to maintain optimal health and fitness.

********************************
